In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
import numpy as np
import os
import time
import imageio
import tensorflow as tf
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, BatchNormalization, LeakyReLU, Dropout, Flatten, Reshape, ReLU

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz

In [ ]:
!tar -xzf 102flowers.tgz
!rm -f 102flowers.tgz

In [ ]:
for infile in glob.glob('/content/jpg/'+'*.jpg'):
  file, ext = os.path.splitext(infile)
  im = Image.open(infile)
  im = im.resize((128,128))
  os.remove(infile)
  im.save(file+'.jpg')

train_data = []
for image_path in glob.glob('/content/jpg/'+'*.jpg'):
  image = imageio.imread(image_path)
  train_data.append(image)

x_train = np.array(train_data)

x_train = (x_train.astype(np.float32) - 127.5) / 127.5

x_train.shape[0]

In [ ]:
plt.imshow(x_train[8188])

In [ ]:
BUFFER_SIZE = x_train.shape[0]
BATCH_SIZE = 256

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def generator_model():
  model = Sequential()
  model.add(Dense(8*8*256, use_bias=False, input_shape=(100,)))
  model.add(BatchNormalization())
  model.add(ReLU())
  model.add(Reshape((8,8,256), input_shape=(8*8*256,)))

  model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', use_bias=False))
  model.add(BatchNormalization())
  model.add(ReLU())

  model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', use_bias=False))
  model.add(BatchNormalization())
  model.add(ReLU())

  model.add(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', use_bias=False))
  model.add(BatchNormalization())
  model.add(ReLU())
      
  model.add(Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

  #model.summary()
  return model

In [ ]:
generator = generator_model()
noise = tf.random.normal([1, 100])
generator_image = generator(noise, training = False)
plt.imshow(generator_image[0, :, :, 0])

In [ ]:
def discriminator_model():
  model = Sequential()
  model.add(Conv2D(32, 5, strides=(2,2), padding='same', input_shape=(128,128,3)))
  model.add(LeakyReLU())
  model.add(Dropout(0.5))

  model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.5))

  model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.5))

  model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(1))
    
  #model.summary()
  return model

In [ ]:
discriminator = discriminator_model()
suspect = discriminator(generator_image)
suspect

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4)

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    print(total_loss)
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
EPOCHS = 5000
noise_dim = 100
num_examples_to_generate = 10
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
from IPython import display

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    display.clear_output(wait=True)
    generate_and_save_images(generator, epoch+1, seed)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  display.clear_output(wait=True)
  generate_and_save_images(generator, epochs, seed)

In [ ]:
%cd '/content/drive/My Drive/GAN1'

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)
  
  fig = plt.figure(figsize=(15,7))
  for i in range(predictions.shape[0]):
      preds = predictions[i,:,:,:]
      preds = np.array(preds)
      preds = (preds * 127.5) + 127.5
      plt.subplot(2, 5, i+1)
      p = (preds).astype(np.uint8)
      plt.axis('off')
  if(epoch % 25 == 0):
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))

In [ ]:
train(train_dataset, EPOCHS)